# Simple API (Application Programming Interface)


### Introduction

An API lets two pieces of software talk to each other. Just like a function,  you don’t have to know how the API works only its inputs and outputs.  An essential type of API is a REST API that allows you to access resources via the internet. In this lab, we will review the Pandas Library in the context of an API, we will also review a basic REST API  


## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">
<li><a href="#ref0">Pandas is an API</a></li>
<li><a href="#ref1">REST APIs Basics  </a></li>


</div>

<hr>


In [ ]:
%pip install pycoingecko
%pip install plotly
%pip install mplfinance
%pip install --upgrade nbformat

<h2 id="PandasAPI">Pandas is an API </h2>


Pandas is actually set of software components , much of  which is not even written in Python.


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

You create a dictionary, this is just data.


In [4]:
dict_ = {'a':[11,21,31],'b':[12,22,32]}

When you create a Pandas object with the Dataframe constructor in API lingo, this is an "instance". The data in the dictionary is passed along to the pandas API. You then use the dataframe to communicate with the API.


In [5]:
df = pd.DataFrame(dict_)
type(df)

pandas.core.frame.DataFrame

<img src="img/api_1.png" width="800," align="center" alt="logistic regression block diagram">


When you call the method head the dataframe communicates with the API displaying the first few rows of the dataframe.




In [7]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


When you call the method mean,the API will calculate the mean and return the value.


In [8]:
df.mean()

a    21.0
b    22.0
dtype: float64

<h2 id="ref1">REST APIs</h2>  
RESTfull (Representational state transfer)


Rest API’s function by sending a request,  the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.

<img src="img/api_2.png" alt="BTC/USD Candlestick Chart" width="800">

Here is a description of the candle sticks.

<img src="img/api_3.png" alt="Candle Stick Definition" width="800" height="600">

In this lab, we will be using the CoinGecko API to create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called <a href='https://github.com/man-c/pycoingecko'>PyCoinGecko</a>. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

Lets start off by getting the data we need. Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.


In [9]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [10]:
type(bitcoin_data)

dict

In [24]:
print(len(bitcoin_data['prices']))

722

In [26]:
bitcoin_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.


In [27]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1695121282809, 27183.994603047813],
 [1695124810966, 27222.58191830418],
 [1695128447573, 27145.730412737612],
 [1695132043410, 27076.486393257255],
 [1695135668460, 27143.212388822172]]

Finally lets turn this data into a Pandas DataFrame.


In [28]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
data.head()

,TimeStamp,Price
0,1695121282809,27183.994603
1,1695124810966,27222.581918
2,1695128447573,27145.730413
3,1695132043410,27076.486393
4,1695135668460,27143.212389


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called <code>Date</code>. We will map our <code>unix_to_datetime</code> to each timestamp and convert it to a readable datetime.


In [42]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.datetime.fromtimestamp(d/1000.0).strftime('%Y-%m-%d'))
data.head()

,TimeStamp,Price,date
0,1695121282809,27183.994603,2023-09-19
1,1695124810966,27222.581918,2023-09-19
2,1695128447573,27145.730413,2023-09-19
3,1695132043410,27076.486393,2023-09-19
4,1695135668460,27143.212389,2023-09-19


Using this modified dataset we can now group by the <code>Date</code> and find the min, max, open, and close for the candlesticks.


In [47]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last', 'count']})
candlestick_data.head()

date         Price                                                
                        min           max         first          last count
0  2023-09-19  27076.486393  27285.481432  27183.994603  27178.377168    11
1  2023-09-20  26936.571053  27318.242558  27136.972845  27076.340775    24
2  2023-09-21  26418.527785  27145.010737  27145.010737  26584.396598    24
3  2023-09-22  26534.177937  26688.337962  26593.195070  26534.177937    24
4  2023-09-23  26536.561163  26615.098926  26563.985677  26587.030719    24

### Buid in Functions in Pandas:

| Aggregate Functions |                   Description                  |
|:-------------------:|:----------------------------------------------:|
| count()             | Returns count for each group                   |
| size()              | Returns size for each group                    |
| sum()               | Returns total sum for each group               |
| mean()              | Returns mean for each group. Same as average() |
| average()           | Returns average for each group. Same as mean() |
| std()               | Returns standard deviation for each group      |
| var()               | Return var for each group                      |
| sem()               | Standard error of the mean of groups           |
| describe()          | Returns different statistics                   |
| min()               | Returns minimum value for each group           |
| max()               | Returns maximum value for each group           |
| first()             | Returns first value for each group             |
| last()              | Returns last value for each group              |
| nth()               | Returns nth value for each group               |

Finally we are now ready to use plotly to create our Candlestick Chart.


In [48]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()